In [2]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
sns.set(style="whitegrid")
%matplotlib inline

In [3]:
#IMPORTING .CSV FILES
airlines = pd.read_csv("C:\Users\Taylor\Desktop/flight-delays1/airlines.csv")
airports = pd.read_csv("C:\Users\Taylor\Desktop/flight-delays1/airports.csv")
flights = pd.read_csv("C:\Users\Taylor\Desktop/flight-delays1/flights.csv")

C:\Users\Taylor\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
flights.shape

(5819079, 31)

In [3]:
#CREATING DUMMY VARIABLES FOR AIRLINES AND MONTHS
dummy_airline = pd.get_dummies(flights['AIRLINE'])
flights = pd.concat([flights, dummy_airline], axis=1)

dummy_dates = pd.get_dummies(flights['MONTH'])
flights = pd.concat([flights, dummy_dates], axis=1)

In [4]:
flights.columns

Index([               u'YEAR',               u'MONTH',                 u'DAY',
               u'DAY_OF_WEEK',             u'AIRLINE',       u'FLIGHT_NUMBER',
               u'TAIL_NUMBER',      u'ORIGIN_AIRPORT', u'DESTINATION_AIRPORT',
       u'SCHEDULED_DEPARTURE',      u'DEPARTURE_TIME',     u'DEPARTURE_DELAY',
                  u'TAXI_OUT',          u'WHEELS_OFF',      u'SCHEDULED_TIME',
              u'ELAPSED_TIME',            u'AIR_TIME',            u'DISTANCE',
                 u'WHEELS_ON',             u'TAXI_IN',   u'SCHEDULED_ARRIVAL',
              u'ARRIVAL_TIME',       u'ARRIVAL_DELAY',            u'DIVERTED',
                 u'CANCELLED', u'CANCELLATION_REASON',    u'AIR_SYSTEM_DELAY',
            u'SECURITY_DELAY',       u'AIRLINE_DELAY', u'LATE_AIRCRAFT_DELAY',
             u'WEATHER_DELAY',                  u'AA',                  u'AS',
                        u'B6',                  u'DL',                  u'EV',
                        u'F9',                  u'HA

In [5]:
#IMPORTING LOGISTIC REGRESSION AND CREATING VARIABLE FOR LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
#WANT TO PREDICT IF A FLIGHT IS CANCELLED OR IF IT IS DELAYED.
#FLIGHT IS CONSIDERED LATE IF IT ARRIVES AT GATE 15 MINUTES AFTER ITS SCHEDULED TIME OF ARRIVAL
flights['DELAYED_15'] = np.where(flights['ARRIVAL_DELAY'] >= 15, 1, 0)
flights.DELAYED_15[flights.DELAYED_15 == 1].count()

1063439

In [7]:
#WANT TO ADD A COLUMN THAT INCLUDES WHETHER A FLIGHT WAS CANCELLED OR DELAYED PAST 15 MINUTES
#CAN DO THIS BY CREATING A LOOP THAT FINDS THIS:
def cancdel(row):
    if row['CANCELLED'] == 1 or row['DELAYED_15'] == 1:
        val = 1
    else:
        val = 0
    return val

In [8]:
#CREATE A NEW DATAFRAME BECAUSE OF POTENTIAL MEMORY ERROR
flights_main = flights[['CANCELLED', 'DELAYED_15']]
flights_main['can_del'] = flights_main.apply(cancdel, axis = 1)

C:\Users\Taylor\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
#CHECK IF CANCELLED + DELAYED = CAN_DEL
print flights_main.CANCELLED[flights_main.CANCELLED == 1].count()
print flights_main.DELAYED_15[flights_main.DELAYED_15 == 1].count()
print flights_main.can_del[flights_main.can_del == 1].count()

89884
1063439
1153323


In [10]:
#SAVE THIS COLUMN INTO FLIGHTS ORIGINAL DATAFRAME
flights['CAN_DEL'] = flights_main['can_del']

In [11]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,5,6,7,8,9,10,11,12,DELAYED_15,CAN_DEL
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0,0,0,0,0,0,0,0,0,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0,0,0,0,0,0,0,0,0,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0,0,0,0,0,0,0,0,0,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0,0,0,0,0,0,0,0,0,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#FIND COLUMNS FOR DUMMY VARIABLES FOR AIRLINES AND MONTHS
print flights.columns[31:45]
print flights.columns[45:57]

Index([u'AA', u'AS', u'B6', u'DL', u'EV', u'F9', u'HA', u'MQ', u'NK', u'OO',
       u'UA', u'US', u'VX', u'WN'],
      dtype='object')
Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='object')


In [13]:
#ACCURACY OF MODEL IS ABOUT THE SAME AS NULL ACCURACY
#RECALL OF THE MODEL IS ALWAYS PREDICTING A FLIGHT WITHOUT CANCELLATION OR DELAY
from sklearn.cross_validation import train_test_split
feature_cols = ['SCHEDULED_DEPARTURE'] + list(flights.columns[31:45]) + list(flights.columns[45:57])
X = flights[feature_cols]
y = flights.CAN_DEL
print X.shape, y.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
logreg.fit(X_train,y_train)    
y_pred_class = logreg.predict(X_test)
print "Null Accuracy: ", float(y.value_counts()[0])/len(y)
print "Accuracy: ", metrics.accuracy_score(y_test, y_pred_class)
print "Recall: ", metrics.recall_score(y_test, y_pred_class)
print metrics.confusion_matrix(y_test, y_pred_class)

C:\Users\Taylor\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(5819079, 27) (5819079L,)
Null Accuracy:  0.801803171945
Accuracy:  0.801642183988
Recall:  0.0
[[1166205       0]
 [ 288565       0]]


In [14]:
#DUE TO THE DATA SET BEING HEAVILY WEIGHTED AGAINST PREDICTING CANCELLATIONS OR DELAYS, WE CAN USE AN UNDERSAMPLING TECHNIQUE
#BALANCE THE DATA SO THAT THERE IS AN EQUAL NUMBER OF CAN_DEL = 0 AND CAN_DEL = 1
flights_original = flights
sample_size = sum(flights_original.CAN_DEL == 1)
CAN_DEL_indices = flights_original[flights_original.CAN_DEL == 1].index
nonCAN_DEL_indices = flights_original[flights_original.CAN_DEL == 0].index
random_indices = np.random.choice(nonCAN_DEL_indices, sample_size, replace=False)

In [15]:
#TEST WHETHER OR NOT CAN_DEL = 0 AND CAN_DEL ARE EQUAL
print "# of on time flights before under sampling: ", flights_original.shape
print "# of cancelled/delayed flights: ", len(CAN_DEL_indices)
flights_samp = flights_original.loc[random_indices]
flights_ones = flights_original.loc[CAN_DEL_indices]
print "# of on time flights after under sampling: ", flights_samp.shape

# of on time flights before under sampling:  (5819079, 59)
# of cancelled/delayed flights:  1153323
# of on time flights after under sampling:  (1153323, 59)


In [16]:
#APPEND CAN_DEL = 0 AND CAN_DEL = 1 INTO ONE DATAFRAME
flights_ones = flights_ones.append(flights_samp)

In [17]:
#CHECK TO SEE THAT NEW DATAFRAME HAS CORRECT COUNTS OF CAN_DEL
flights_ones.shape
print flights_ones.CAN_DEL.value_counts()

1    1153323
0    1153323
Name: CAN_DEL, dtype: int64


In [18]:
#CHECK TO SEE THAT NEW DATAFRAME HAS SAME COLUMNS
flights_ones.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,5,6,7,8,9,10,11,12,DELAYED_15,CAN_DEL
27,2015,1,1,4,NK,597,N528NK,MSP,FLL,115,...,0,0,0,0,0,0,0,0,1,1
30,2015,1,1,4,NK,168,N629NK,PHX,ORD,125,...,0,0,0,0,0,0,0,0,1,1
32,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,0,0,0,0,0,0,0,0,0,1
35,2015,1,1,4,HA,17,N389HA,LAS,HNL,145,...,0,0,0,0,0,0,0,0,1,1
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,200,...,0,0,0,0,0,0,0,0,0,1


In [19]:
#USE NEW DATAFRAME FOR PREDICTING ACCURACY AND RECALL
feature_cols = ['SCHEDULED_DEPARTURE'] + list(flights_ones.columns[31:45]) + list(flights_ones.columns[45:57])
X = flights_ones[feature_cols]
y = flights_ones.CAN_DEL
print X.shape, y.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
logreg.fit(X_train,y_train)    
y_pred_class = logreg.predict(X_test)
print "Null Accuracy: ", float(y.value_counts()[0])/len(y)
print "Accuracy: ", metrics.accuracy_score(y_test, y_pred_class)
print "Recall: ", metrics.recall_score(y_test, y_pred_class)
print metrics.confusion_matrix(y_test, y_pred_class)

(2306646, 27) (2306646L,)
Null Accuracy:  0.5
Accuracy:  0.602637246775
Recall:  0.611800563056
[[171276 117315]
 [111829 176242]]
